In [1]:
import numpy as np 
import pandas as pd 
import keras.losses
import re

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, Activation, SpatialDropout1D
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
from keras.backend import clear_session

In [2]:
df=pd.read_csv('train.csv')
df=df.dropna()
df.head(10)

textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   
5  28b57f3990  http://www.dothebouncy.com/smf - some shameles...   
6  6e0c6d75b1  2am feedings for the baby are fun when he is a...   
7  50e14c0bb8                                         Soooo high   
8  e050245fbd                                        Both of you   
9  fc2cbefa9d   Journey!? Wow... u just became cooler.  hehe....   

                                       selected_text sentiment  
0                I`d have responded, if I were going   neutral  
1                                           Sooo SAD  negative  
2                                        bullying me  negative  
3                                     leave me alone  negative  
4                                      Sons of ****,  negative  
5  http://www.dothebouncy.com/smf - some shameles...   neutral  
6                                                fun  positive  
7                                         Soooo high   neutral  
8                                        Both of you   neutral  
9                       Wow... u just became cooler.  positive

In [3]:
df.describe

<bound method NDFrame.describe of            textID                                               text  \
0      cb774db0d1                I`d have responded, if I were going   
1      549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2      088c60f138                          my boss is bullying me...   
3      9642c003ef                     what interview! leave me alone   
4      358bd9e861   Sons of ****, why couldn`t they put them on t...   
...           ...                                                ...   
27476  4eac33d1c0   wish we could come see u on Denver  husband l...   
27477  4f4c4fc327   I`ve wondered about rake to.  The client has ...   
27478  f67aae2310   Yay good for both of you. Enjoy the break - y...   
27479  ed167662a5                         But it was worth it  ****.   
27480  6f7127d9d7     All this flirting going on - The ATG smiles...   

                                           selected_text sentiment  
0                    I`d have re

In [4]:
df.info

<bound method DataFrame.info of            textID                                               text  \
0      cb774db0d1                I`d have responded, if I were going   
1      549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2      088c60f138                          my boss is bullying me...   
3      9642c003ef                     what interview! leave me alone   
4      358bd9e861   Sons of ****, why couldn`t they put them on t...   
...           ...                                                ...   
27476  4eac33d1c0   wish we could come see u on Denver  husband l...   
27477  4f4c4fc327   I`ve wondered about rake to.  The client has ...   
27478  f67aae2310   Yay good for both of you. Enjoy the break - y...   
27479  ed167662a5                         But it was worth it  ****.   
27480  6f7127d9d7     All this flirting going on - The ATG smiles...   

                                           selected_text sentiment  
0                    I`d have resp

In [5]:
df.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [6]:
df = df[['selected_text','sentiment']]
df

selected_text sentiment
0                    I`d have responded, if I were going   neutral
1                                               Sooo SAD  negative
2                                            bullying me  negative
3                                         leave me alone  negative
4                                          Sons of ****,  negative
...                                                  ...       ...
27476                                             d lost  negative
27477                                      , don`t force  negative
27478                          Yay good for both of you.  positive
27479                         But it was worth it  ****.  positive
27480  All this flirting going on - The ATG smiles. Y...   neutral

[27480 rows x 2 columns]

In [7]:
df = df[df.sentiment != "neutral"]
df['selected_text'] = df['selected_text'].astype(str).apply(lambda x: x.lower())
df['selected_text'] = df['selected_text'].apply((lambda x:re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in df.iterrows():
  row[0]=row[0].replace('rt',' ')

In [8]:
max_features = 10000
tokenizer = Tokenizer(num_words = max_features,split=' ')
tokenizer.fit_on_texts(df['selected_text'].values)
x=tokenizer.texts_to_sequences(df['selected_text'].values)
x=pad_sequences(x)

In [9]:
y=pd.get_dummies(df['sentiment']).values

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
clear_session()
print("shape of X_train: ",x_train.shape)
print("shape of X_test: ",x_test.shape)

print("shape of y_train: ",y_train.shape)
print("shape of y_test: ",y_test.shape)

shape of X_train:  (13090, 31)
shape of X_test:  (3273, 31)
shape of y_train:  (13090, 2)
shape of y_test:  (3273, 2)


In [11]:
embed_dim = 128

model = Sequential(name="Simple_RNN")
model.add(Embedding(max_features,embed_dim))
model.add(SimpleRNN(128,activation='tanh',return_sequences=False))
model.add(Dense(2, activation='softmax'))


# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.compile(loss='binary_crossentropy',optimizer='adadelta',metrics=['accuracy'])
# model.compile(loss='binary_crossentropy',optimizer='adagrad',metrics=['accuracy'])
# model.compile(loss='binary_crossentropy',optimizer='SGD',metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['accuracy'])



print(model.summary())

Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 1,313,154
Trainable params: 1,313,154
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
batch_size = 56
model.fit(x_train, y_train, epochs = 3, batch_size = batch_size, verbose = 2)

Epoch 1/3
234/234 - 6s - loss: 0.4107 - accuracy: 0.8063 - 6s/epoch - 26ms/step
Epoch 2/3
234/234 - 4s - loss: 0.2270 - accuracy: 0.9204 - 4s/epoch - 19ms/step
Epoch 3/3
234/234 - 4s - loss: 0.1803 - accuracy: 0.9425 - 4s/epoch - 19ms/step


In [13]:
model.evaluate(x_test, y_test, batch_size = batch_size)

59/59 [==============================] - 1s 7ms/step - loss: 0.2496 - accuracy: 0.8854


[0.2496286779642105, 0.8854262232780457]

In [14]:
model.save('SimpleRNN.h5')